## Импорты

In [72]:
import os
import random
from copy import deepcopy
from pprint import pprint

import numpy as np
import pandas as pd

from IPython.display import display
from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import MAP, NDCG, MeanInvUserFreq, Precision, Recall, Serendipity
from rectools.metrics.base import MetricAtK
from rectools.model_selection import Splitter, TimeRangeSplitter
from rectools.models import PopularModel, RandomModel
from rectools.models.base import ModelBase
from tqdm import tqdm

from helpers.metrics import calculate_metrics
from helpers.visualization import visualize_metrics, visualize_training_result

Конфиг

In [73]:
RANDOM_STATE = 32 # из задания
random.seed(RANDOM_STATE)
os.environ["PYTHONHASHSEED"] = str(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

K_RECOS = 10 # Кол-во рекоммендаций
N_SPLITS = 3 # Фолды

## Датасет

Данные предварительно предзагружены в kion_train вручную. Загрузим данные

In [74]:
users = pd.read_csv("kion_train/users.csv")
items = pd.read_csv("kion_train/items.csv")
interactions = pd.read_csv("kion_train/interactions.csv", parse_dates=["last_watch_dt"])

Переименуем столбцы для rectools: last_watch_dt в datetime; total_dur в weight. Используем маппинг из Columns.

In [75]:
interactions.rename(columns={"last_watch_dt": Columns.Datetime, "total_dur": Columns.Weight}, inplace=True)
interactions = Interactions(interactions)

Посмотрим что в interactions

In [76]:
interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


## Подсчет метрик

Инициализируем модели

In [77]:
models: list[ModelBase] = [
    RandomModel(random_state=RANDOM_STATE),
    PopularModel()
]

Инициализиуем метрики по порогам 1, 5, 10

In [78]:
metrics: dict[str, MetricAtK] = {}
for k in [1, 5, 10]:
    metrics.update(
        {
            f"top@{k}_precision": Precision(k=k),
            f"top@{k}_recall": Recall(k=k),
            f"top@{k}_ndcg": NDCG(k=k),
            f"top@{k}_map": MAP(k=k),
            f"top@{k}_serendipity": Serendipity(k=k),
            f"top@{k}_mean_inv_user_freq": MeanInvUserFreq(k=k),
        }
    )

Инициализиурем TimeRangeSplitter

In [79]:
splitter: Splitter = TimeRangeSplitter(
    test_size="7D",
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

Выполним подсчет метрик

In [80]:
model_metrics = []
for model in tqdm(models, total=len(models)):
    model_metrics.extend(calculate_metrics(interactions=interactions, metrics=metrics, model=model, splitter=splitter, k_recos=K_RECOS))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [04:37<00:00, 138.51s/it]


Визуализируем метрики

In [81]:
visualize_metrics(model_metrics)

## Визуализация рекомендаций

Визуализация рекомендаций и историй просмотров для юзеров [666262, 672861, 955527] (из задания)

In [82]:
user_ids = [666262, 672861, 955527]

In [83]:
dataset = Dataset.construct(interactions.df)
item_data = items[["item_id", "title", "genres"]]

Посчитаем и отобразим результат

In [84]:
reports = {}
for model in models:
    init_model = deepcopy(model)
    init_model.fit(dataset)
    
    result = visualize_training_result(
        model=init_model, dataset=dataset, user_ids=user_ids, item_data=item_data, k_recos=K_RECOS, interactions=interactions
    )

    pprint(f"Model: {model.__class__}")
    display(result)

"Model: <class 'rectools.models.random.RandomModel'>"


"Model: <class 'rectools.models.popular.PopularModel'>"
